# Convert h5 model to trt model

In [9]:
import sys 
import os

NOMEROFF_NET_DIR = "../../"

# setup result model dirs
SAVED_MODEL_DIR  = os.path.join(NOMEROFF_NET_DIR, "./models/saved_model_ocr_ru")
TRT_SAVED_MODEL_DIR  = os.path.join(NOMEROFF_NET_DIR, "./models/trt_saved_model_ocr_ru")

sys.path.append(NOMEROFF_NET_DIR)

In [10]:
import tensorflow as tf
import matplotlib.image as mpimg

# dynamically grow the memory used on the GPU
from tensorflow.compat.v1.keras.backend import set_session
config = tf.compat.v1.ConfigProto()
config.gpu_options.allow_growth = True 
sess = tf.compat.v1.Session(config=config)
set_session(sess)

from NomeroffNet import  TextDetector

## load latest h5 model

In [11]:
textDetector = TextDetector.get_static_module("ru")()
textDetector.load("latest")

In [12]:
textDetector.MODEL.summary()

Model: "functional_1"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
the_input_ru (InputLayer)       [(None, 128, 64, 1)] 0                                            
__________________________________________________________________________________________________
conv1 (Conv2D)                  (None, 128, 64, 16)  160         the_input_ru[0][0]               
__________________________________________________________________________________________________
max1 (MaxPooling2D)             (None, 64, 32, 16)   0           conv1[0][0]                      
__________________________________________________________________________________________________
conv2 (Conv2D)                  (None, 64, 32, 16)   2320        max1[0][0]                       
_______________________________________________________________________________________

## convert to pb model

In [13]:
tf.saved_model.save(textDetector.MODEL, SAVED_MODEL_DIR)

Instructions for updating:
This property should not be used in TensorFlow 2.0, as updates are applied automatically.
Instructions for updating:
This property should not be used in TensorFlow 2.0, as updates are applied automatically.
INFO:tensorflow:Assets written to: ../.././models/saved_model_ocr_ru/assets


## convert pb model to trt model

In [6]:
params = tf.experimental.tensorrt.ConversionParams(
    minimum_segment_size=50,
    max_workspace_size_bytes=1 << 25,
    precision_mode  = "FP16",
)

converter = tf.experimental.tensorrt.Converter(
    input_saved_model_dir=SAVED_MODEL_DIR,
    conversion_params=params)
converter.convert()
converter.save(TRT_SAVED_MODEL_DIR)

INFO:tensorflow:Linked TensorRT version: (7, 1, 3)
INFO:tensorflow:Loaded TensorRT version: (7, 1, 3)
INFO:tensorflow:Assets written to: ../.././models/trt_saved_model_ocr_ru/assets


## run trt model

In [7]:
saved_model_loaded = tf.saved_model.load(TRT_SAVED_MODEL_DIR)

In [8]:
import matplotlib.image as mpimg
import numpy as np

img_path = '../../examples/crop_np_images/JJF509.png'
imgs = [mpimg.imread(img_path)]
Xs = []
for img in imgs:
    Xs.append(textDetector.normalize_pb(img[:,:,:3]))

In [9]:
f = saved_model_loaded.signatures["serving_default"]

In [10]:
tensorX = tf.convert_to_tensor(np.array(Xs).astype(np.float32))

In [11]:
f(tensorX)["softmax_ru"]

<tf.Tensor: shape=(1, 32, 23), dtype=float32, numpy=
array([[[2.51578103e-06, 1.08354030e-07, 9.96366634e-06, 2.05394349e-06,
         4.04694305e-07, 5.88475359e-07, 1.99150691e-07, 5.36061293e-07,
         3.87997978e-04, 3.13864148e-04, 1.44897320e-04, 9.97589111e-01,
         2.64340798e-07, 1.91849426e-06, 2.94388265e-05, 1.81083306e-06,
         3.69691174e-04, 1.34698973e-06, 7.70275597e-04, 1.39842018e-06,
         9.24338965e-05, 4.87835496e-06, 2.74338934e-04],
        [3.23916718e-12, 1.98988720e-13, 7.19061616e-12, 3.22991146e-13,
         4.76196209e-12, 1.94519422e-12, 1.98223138e-13, 2.44324519e-14,
         3.74683005e-08, 2.20891625e-08, 1.42103545e-06, 9.99997020e-01,
         2.51848824e-12, 5.46168215e-11, 1.57246589e-08, 7.86794310e-11,
         1.40993245e-06, 3.13284976e-09, 3.85613461e-08, 4.33671632e-09,
         9.32788211e-08, 1.55251402e-11, 3.59701291e-09],
        [2.34548600e-13, 2.93949815e-13, 4.83682972e-12, 1.17348310e-13,
         3.99973075e-11, 8.3